# Importing Libraries

In [4]:
import os
import re
from scipy import ndimage, misc
from skimage.transform import resize, rescale
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

np.random.seed(0)

from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.layers import Conv2DTranspose, UpSampling2D, add
from tensorflow.keras.models import Model
from tensorflow.keras import regularizers
import tensorflow as tf

# Encoder Model

In [5]:
input_img = Input(shape=(256,256,3))
layer1 = Conv2D(64, (3,3), padding ='same', activation='relu',
                activity_regularizer = regularizers.l1(10e-10))(input_img)
layer2 = Conv2D(64, (3,3), padding ='same', activation='relu',
                activity_regularizer = regularizers.l1(10e-10))(layer1)
layer3 = MaxPooling2D(padding='same')(layer2)
layer4 = Conv2D(128, (3,3), padding ='same', activation='relu',
                activity_regularizer = regularizers.l1(10e-10))(layer3)
layer5 = Conv2D(128, (3,3), padding ='same', activation='relu',
                activity_regularizer = regularizers.l1(10e-10))(layer4)
layer6 = MaxPooling2D(padding='same')(layer5)
layer7 = Conv2D(256, (3,3), padding ='same', activation='relu',
                activity_regularizer = regularizers.l1(10e-10))(layer6)
encoder = Model(input_img, layer7)

In [8]:
encoder.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 256, 256, 64)      1792      
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 256, 256, 64)      36928     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 128, 128, 64)      0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 128, 128, 128)     73856     
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 128, 128, 128)     147584    
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 64, 64, 128)       0     

# Decoder Model

In [22]:
# Encoder Layers
input_img = Input(shape=(256,256,3))
layer1 = Conv2D(64, (3,3), padding ='same', activation='relu',
                activity_regularizer = regularizers.l1(10e-10))(input_img)
layer2 = Conv2D(64, (3,3), padding ='same', activation='relu',
                activity_regularizer = regularizers.l1(10e-10))(layer1)
layer3 = MaxPooling2D(padding='same')(layer2)
layer4 = Conv2D(128, (3,3), padding ='same', activation='relu',
                activity_regularizer = regularizers.l1(10e-10))(layer3)
layer5 = Conv2D(128, (3,3), padding ='same', activation='relu',
                activity_regularizer = regularizers.l1(10e-10))(layer4)
layer6 = MaxPooling2D(padding='same')(layer5)
layer7 = Conv2D(256, (3,3), padding ='same', activation='relu',
                activity_regularizer = regularizers.l1(10e-10))(layer6)

In [23]:
# Decoder Layers
layer8 = UpSampling2D()(layer7)
layer9 = Conv2D(128, (3,3), padding ='same', activation = 'relu',
               activity_regularizer=regularizers.l1(10e-10))(layer8)
layer10 = Conv2D(128, (3,3), padding ='same', activation = 'relu',
               activity_regularizer=regularizers.l1(10e-10))(layer9)
        # merge Layer
layer11 = add([layer5, layer10])
layer12 = UpSampling2D()(layer11)
layer13 = Conv2D(64, (3,3), padding ='same', activation = 'relu',
               activity_regularizer=regularizers.l1(10e-10))(layer12)
layer14 = Conv2D(64, (3,3), padding ='same', activation = 'relu',
               activity_regularizer=regularizers.l1(10e-10))(layer13)
layer15 = add([layer14, layer2])
decoder = Conv2D(3, (3,3), padding ='same', activation = 'relu',
               activity_regularizer=regularizers.l1(10e-10))(layer15)
autoencoder = Model(input_img, decoder)
autoencoder.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv2d_35 (Conv2D)              (None, 256, 256, 64) 1792        input_6[0][0]                    
__________________________________________________________________________________________________
conv2d_36 (Conv2D)              (None, 256, 256, 64) 36928       conv2d_35[0][0]                  
__________________________________________________________________________________________________
max_pooling2d_10 (MaxPooling2D) (None, 128, 128, 64) 0           conv2d_36[0][0]                  
____________________________________________________________________________________________

In [20]:
autoencoder.compile(optimizer='adadelta', loss='mean_squared_error')

# Note:

#### Model is not trained due to computational limitations

Go to <a href="https://towardsdatascience.com/image-super-resolution-using-convolution-neural-networks-and-auto-encoders-28c9eceadf90">Link</a> to download the dataset and code for training the model.